<a href="https://colab.research.google.com/github/aapoorv-tf/abs-summarizer-using-seq2seq/blob/main/seq2seq_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.insert(0,'/content/gdrive/MyDrive/IR Project')

In [ ]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
cnn = tfds.load(name="cnn_dailymail")
cnn = tfds.as_dataframe(cnn)

In [ ]:
import tensorflow as tf
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')
 
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.50.83.18:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.50.83.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.50.83.18:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
cnnt = pd.read_csv("/content/gdrive/MyDrive/IR Project/Dataset/CDataset/cnn_dailymail_train.csv", nrows=90000)
cnnv = pd.read_csv("/content/gdrive/MyDrive/IR Project/Dataset/CDataset/cnn_dailymail_val.csv", nrows=10000)

In [ ]:
data = [cnnt,cnnv]
data = pd.concat(data)

In [ ]:
data.head()

,Unnamed: 0,article,highlights
0,0,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown..."
1,1,"(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, author...","Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murde..."
2,2,"A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he r...","Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titl..."
3,3,"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prio...","Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says ..."
4,4,Fleetwood are the only team still to have a 100% record in Sky Bet League One as a 2-0 win over Scunthorpe sent Graham Alexander’s men top of the table. The Cod Army are playing in the third tier ...,"Fleetwood top of League One after 2-0 win at Scunthorpe .\nPeterborough, Bristol City, Chesterfield and Crawley all drop first points of the season .\nStand-in striker Matt Done scores a hat-trick..."


In [ ]:
data.shape

(100000, 3)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  100000 non-null  int64 
 1   article     100000 non-null  object
 2   highlights  100000 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

stop_words = set(stopwords.words('english'))
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    return (" ".join(tokens)).strip()

In [ ]:
cleaned_article = []
for t in data['article']:
    cleaned_article.append(text_cleaner(t))

In [ ]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    return newString

In [ ]:
cleaned_summary = []
for t in data['highlights']:
    cleaned_summary.append(summary_cleaner(t))

In [ ]:
data['cleaned_article'] = cleaned_article
data['cleaned_summary'] = cleaned_summary

In [ ]:
data.head()

,Unnamed: 0,article,highlights,cleaned_article,cleaned_summary
0,0,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown...",associated press published est october updated est october bishop fargo catholic diocese north dakota exposed potentially hundreds church members fargo grand forks jamestown hepatitis virus late s...,bishop john folda of north dakota is taking time off after being diagnosed he contracted the infection through contaminated food in italy church members in fargo grand forks and jamestown c...
1,1,"(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, author...","Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murde...",ralph mata internal affairs lieutenant miami dade police department working division investigates allegations wrongdoing cops outside office authorities allege year old longtime officer worked dru...,criminal complaint cop used his role to help cocaine traffickers ralph mata an internal affairs lieutenant allegedly helped group get guns he also arranged to pay two assassins in a murder ...
2,2,"A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he r...","Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titl...",drunk driver killed young woman head crash checking mobile phone jailed six years craig eccleston todd driving home night pub received text message reading replying veered across road driving roun...,craig eccleston todd had drunk at least three pints before driving car was using phone when he veered across road in yarmouth isle of wight crashed head on into year old rachel titley...
3,3,"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prio...","Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says ...",breezy sweep pen president vladimir putin wrote new chapter crimea turbulent history committing region future returned russian domain sixty years prior ukraine breakaway peninsula signed away swif...,nina dos santos says europe must be ready to accept sanctions will hurt both sides targeting russia business community would be one way of sapping their support for president putin she says b...
4,4,Fleetwood are the only team still to have a 100% record in Sky Bet League One as a 2-0 win over Scunthorpe sent Graham Alexander’s men top of the table. The Cod Army are playing in the third tier ...,"Fleetwood top of League One after 2-0 win at Scunthorpe .\nPeterborough, Bristol City, Chesterfield and Crawley all drop first points of the season .\nStand-in striker Matt Done scores a hat-trick...",fleetwood team still record sky bet league one win scunthorpe sent graham alexander men top table cod army playing third tier first time history six promotions nine years remarkable ascent shows s...,fleetwood top of league one after win at scunthorpe peterborough bristol city chesterfield and crawley all drop first points of the

In [ ]:
wc = pd.DataFrame()
wc['article_count'] = data['cleaned_article'].apply(lambda x: len(str(x).split()))
for i in range(90,100):
  var = wc['article_count'].values
  var = np.sort(var,axis=None)
  print('At {} percentile, word count is {}'.format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile - ",var[-1])

At 90 percentile, word count is 631
At 91 percentile, word count is 648
At 92 percentile, word count is 668
At 93 percentile, word count is 688
At 94 percentile, word count is 712
At 95 percentile, word count is 738
At 96 percentile, word count is 768
At 97 percentile, word count is 804
At 98 percentile, word count is 851
At 99 percentile, word count is 910
100 percentile -  1934


In [ ]:
wc = pd.DataFrame()
wc['summary_count'] = data['cleaned_summary'].apply(lambda x: len(str(x).split()))
for i in range(90,100):
  var = wc['summary_count'].values
  var = np.sort(var,axis=None)
  print('At {} percentile, word count is {}'.format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile - ",var[-1])

At 90 percentile, word count is 73
At 91 percentile, word count is 75
At 92 percentile, word count is 77
At 93 percentile, word count is 79
At 94 percentile, word count is 82
At 95 percentile, word count is 85
At 96 percentile, word count is 89
At 97 percentile, word count is 94
At 98 percentile, word count is 101
At 99 percentile, word count is 113
100 percentile -  1208


In [ ]:
max_len_art = 700
max_len_summ = 75

In [ ]:
cleaned_article =np.array(data['cleaned_article'])
cleaned_summary=np.array(data['cleaned_summary'])

short_article=[]
short_summary=[]

for i in range(len(cleaned_article)):
    if(len(cleaned_summary[i].split())<=max_len_summ and len(cleaned_article[i].split())<=max_len_art):
        short_article.append(cleaned_article[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'article':short_article,'summary':short_summary})

In [ ]:
df['summary'] = df['summary'].apply(lambda x: 'sostok '+ x +' eostok')

In [ ]:
df.tail()

,article,summary
86505,woman arrested suspicion drunk aircraft allegedly stripped flight jamaica exposed year old woman south west london arrested police british airways flight landed gatwick airport yesterday kingston ...,sostok year old woman from south west london arrested on arrival at gatwick woman was drunk and had stripped off on flight from jamaica the woman caused disruption after performing a solo s...
86506,fundraiser pregnant widow whose husband killed suddenly tragic snowmobile accident earned space five days jeffrey kinn year old metal worker greenville new york life insurance died last thursday h...,sostok jeffrey kinn a year old metal worker from greenville was killed in a head on collision between two snowmobiles his wife tyler kinn is six months pregnant with their first child ...
86507,peacocks birds paradise male birds well known showing brightly coloured plumage way attracting mate however new research suggested males birds may actually growing less distinct female counterpart...,sostok study was conducted by researchers at university of wisconsin milwaukee they measured the male and female plumage colour of bird species the researchers found some species show diff...
86508,year old woman regular brushes death one year pronounced clinically dead times sara brautigam four years ago diagnosed postural orthostatic tachycardia syndrome condition makes heart race standing...,sostok the year old has regular brushes with death due to rare heart condition condition means she has palpitations that cause her heart to stop beating in one year alone ms brautigam was p...
86509,chilean girl plea via social media assisted suicide caught attention person teen thinks could make happen president michelle bachelet bachelet visited year old valentina maureira saturday girl vid...,sostok year old valentina maureira suffers from cystic fibrosis in a video she asks the president to allow her to have an assisted suicide eostok


In [ ]:
df.shape

(86510, 2)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(np.array(df['article']),np.array(df['summary']),test_size=0.3,random_state=0,shuffle=True)

In [ ]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_train))

#rare words coverage for article
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


% of rare words in vocabulary: 54.28225305657074
Total Coverage of rare words: 0.7810724568042784


In [ ]:
x_tokenizer = Tokenizer(num_words = tot_cnt-cnt)
x_tokenizer.fit_on_texts(list(x_train))

x_train    =   x_tokenizer.texts_to_sequences(x_train) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

x_train    =   pad_sequences(x_train,  maxlen=max_len_art, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_art, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [ ]:
x_voc_size

196463

In [ ]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train))

# rare words coverage in summary
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 61.98465231811055
Total Coverage of rare words: 2.381947225800399


In [ ]:
y_tokenizer = Tokenizer(num_words = tot_cnt - cnt)
y_tokenizer.fit_on_texts(list(y_train))

y_train    =   y_tokenizer.texts_to_sequences(y_train) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 

y_train    =   pad_sequences(y_train, maxlen=max_len_summ, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summ, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

In [ ]:
y_voc_size

72064

In [ ]:
y_tokenizer.word_counts['sostok'],len(y_train)

(60557, 60557)

In [ ]:
#deleting empty rows
ind=[]
for i in range(len(y_train)):
    cnt=0
    for j in y_train[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_train=np.delete(y_train,ind, axis=0)
x_train=np.delete(x_train,ind, axis=0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [ ]:
from keras import backend as K
K.clear_session()

latent_dim=300
embedding_dim=100

def seq2seq():
  
  global encoder_inputs
  global enc_emb
  global encoder_outputs
  global state_h
  global state_c
  global decoder_inputs
  global dec_emb_layer
  global decoder_lstm
  global decoder_dense
  encoder_inputs = Input(shape=(max_len_art,))
  #embedding layer
  
  enc_emb =  Embedding(x_voc_size, embedding_dim,trainable=True)(encoder_inputs)
  #encoder lstm 1
  encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
  encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

  #encoder lstm 2
  encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
  encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

    #encoder lstm 3
  encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
  
  encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)
  
  decoder_inputs = Input(shape=(None,))
  #embedding layer
  
  dec_emb_layer = Embedding(y_voc_size, embedding_dim,trainable=True)
  dec_emb = dec_emb_layer(decoder_inputs)

  
  decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
  decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

  #dense layer
  
  decoder_dense =  TimeDistributed(Dense(y_voc_size, activation='softmax'))
  decoder_outputs = decoder_dense(decoder_outputs)

  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  return model


In [ ]:
with tpu_strategy.scope():
  model1=seq2seq()
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 700)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 700, 100)     19646300    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 700, 300), ( 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras import optimizers
nadam = optimizers.Nadam(lr=0.0001,
                         beta_1=0.9,
                         beta_2=0.999,
                         epsilon=1e-8, 
                         schedule_decay=0.004)
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer=nadam,
              metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [ ]:
history=model1.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:],epochs=10,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/10
474/474 [==============================] - 898s 2s/step - loss: 5.9118 - acc: 0.4051 - val_loss: 4.6056 - val_acc: 0.4136
Epoch 2/10
474/474 [==============================] - 827s 2s/step - loss: 4.5270 - acc: 0.4195 - val_loss: 4.4099 - val_acc: 0.4368
Epoch 3/10
474/474 [==============================] - 827s 2s/step - loss: 4.4317 - acc: 0.4363 - val_loss: 4.3568 - val_acc: 0.4419
Epoch 4/10
474/474 [==============================] - 827s 2s/step - loss: 4.3719 - acc: 0.4425 - val_loss: 4.2949 - val_acc: 0.4498
Epoch 5/10
474/474 [==============================] - 827s 2s/step - loss: 4.3182 - acc: 0.4471 - val_loss: 4.2568 - val_acc: 0.4514
Epoch 6/10
474/474 [==============================] - 827s 2s/step - loss: 4.2887 - acc: 0.4488 - val_loss: 4.2341 - val_acc: 0.4538
Epoch 7/10
474/474 [==============================] - 827s 2s/step - loss: 4.2647 - acc: 0.4506 - val_loss: 4.2126 - val_acc: 0.4547
Epoch 8/10
474/474 [==============================] - 827s 2s/step - 

In [ ]:
model1.save("/content/gdrive/MyDrive/IR Project/seq2seq_batch_128_epochs_10.h5")  

In [ ]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [ ]:

encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                      state_h, state_c])


decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_len_art, latent_dim))

dec_emb2 = dec_emb_layer(decoder_inputs)

(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
        initial_state=[decoder_state_input_h, decoder_state_input_c])

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                      decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    7206400     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 300)]        0                                            
____________________________________________________________________________________________

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index+1]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_len_summ-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString


In [ ]:
for i in range(0,10):
    print("Article:",seq2text(x_val[i]))
    print("Original summary:",seq2summary(y_val[i]))
    print("Predicted summary:",decode_sequence(x_val[i].reshape(1,max_len_art)))
    print("\n")

In [ ]:
e_out, e_h, e_c = encoder_model.predict([x_val[2].reshape(1,max_len_art)])
e_out

array([[[-2.5858199e-03, -2.5425670e-03, -8.3016261e-04, ...,
         -3.4304427e-03, -4.2542391e-03,  2.1860003e-03],
        [-5.8229445e-03, -5.0611659e-03, -1.9341446e-03, ...,
         -7.2180540e-03, -7.3330533e-03,  4.3918449e-03],
        [-7.9106372e-03, -6.3761724e-03, -2.8139290e-03, ...,
         -1.0367798e-02, -8.5317157e-03,  5.2703316e-03],
        ...,
        [-9.8920590e-01, -9.4010639e-01, -3.1490810e-02, ...,
         -9.4217014e-01, -1.2793975e-01,  9.6344310e-01],
        [-9.8920590e-01, -9.4010639e-01, -3.1490810e-02, ...,
         -9.4217014e-01, -1.2793975e-01,  9.6344316e-01],
        [-9.8920590e-01, -9.4010639e-01, -3.1490810e-02, ...,
         -9.4217014e-01, -1.2793975e-01,  9.6344316e-01]]], dtype=float32)

In [ ]:
np.zeros((1,1))

array([[0.]])

In [ ]:
target_seq

array([[4.]])

In [ ]:
target_seq = np.zeros((1,1))
target_seq[0,0] = target_word_index['sostok']
ot,h,c = decoder_model.predict([target_seq] + [e_out,e_h,e_c])
ot

array([[[5.6082464e-04, 7.4542157e-02, 2.2772681e-03, ...,
         5.8579314e-08, 5.9814781e-08, 5.9788263e-08]]], dtype=float32)

In [ ]:
ot[0,-1,:]

array([5.6082261e-04, 7.4542493e-02, 2.2772676e-03, ..., 5.8579186e-08,
       5.9814766e-08, 5.9788128e-08], dtype=float32)

In [ ]:
sampled_token_index = np.argmax(ot[0, -1, :])
sampled_token_index

1